In [ ]:
.libPaths("/lustre/groups/itg/teams/zeggini/projects/T2D-diamante/T2DGGI_MR/Rpackages")
pathMR="/lustre/groups/itg/teams/zeggini/projects/T2D-diamante/T2DGGI_MR/"
pathMR_res=paste0(pathMR, "multi_omics_github/results/ResultsMR/")
setwd(pathMR)
##Need to include the following paths with the data:
pathMR_ARIC_EA
pathMR_ARIC_AA
pathMR_GENOA_AA
pathMR_GENOA_EA
pathMR_SAGE_AA
pathMR_eQTLGen
path_T2DGGI
pathMR_GALAII_PR
pathMR_GALAII_MX
path_eQTLGen
path_UKBB_pQTL_EUR
path_UKBB_pQTL_AFR
path_UKBB_pQTL_EAS
path_TIGER_eQTL_EUR

path_code #Where are the scripts to be used
path_Borges = #Path with the data from Borges et al. 2022

pwcoco.path #Path where pwcoco is installed
ref.bfile.path #Path to 1000 Genome project bfiles

In [ ]:
library(data.table)
library(parallel)
library(magrittr)
library(coloc)

# datasets

In [ ]:
##Dataframe with the different datasets to use
info.datasets.forMR <- data.frame(STUDY = c("ARIC", "ARIC", "GENOA", "GENOA", "UKB", "GALAII", "GALAII", "SAGE", "eQTLGen", "ROSMAP", "ROSMAP", "UKBB", "UKBB", "UKBB", "TIGER"),
                                 TISSUE = c("plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "plasma", "brain", "brain", "plasma", "plasma", "plasma", "PancreaticIslets"),
                                 BUILD = c("38", "38", "37", "37", "37", "38", "38", "38", "37", "37", "37", "38", "38", "38", "37"),
                                 QTL_TYPE = c("pQTL", "pQTL", "eQTL", "eQTL", "metaboQTL", "eQTL", "eQTL", "eQTL", "eQTL", "pQTL", "eQTL", "pQTL", "pQTL", "pQTL", "eQTL"),
                                 CORRECTION = c("fdr", "fdr", NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,NA),
                                  #Here we put 1 as pval thr for GALAII as results have already been filtered, same for eQTLGen and ROSMAP and UKBB (supplementary with significant downloaded), and TIGER
                                 PVAL_THR = c(0.05, 0.05, 5e-4, 5e-4, 5e-8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
                                 ANCESTRY = c("EA", "AA", "AA", "EA", "EUR", "PR", "MX", "AA", "EUR", "EUR", "EUR", "EUR", "EAS", "AFR", "EUR"),
                                 ANCESTRY_T2D = c("EUR", "AFR", "AFR", "EUR", "EUR", "AMR", "AMR", "AFR", "EUR", "EUR", "EUR", "EUR", "EAS", "AFR", "EUR"),
                                 N = c(7213, 1871, 1032, 801, 115078, 893, 784, 757, 31684, 716, 589, 34557, 262, 931, 404),
                                 PATH_QTL = c(pathMR_ARIC_EA, pathMR_ARIC_AA, pathMR_GENOA_AA, pathMR_GENOA_EA, path_Borges, pathMR_GALAII_PR, pathMR_GALAII_MX, pathMR_SAGE_AA, path_eQTLGen, path_ROSMAP_pQTL, path_ROSMAP_eQTL, path_UKBB_pQTL_EUR, path_UKBB_pQTL_EAS, path_UKBB_pQTL_AFR, path_TIGER_eQTL_EUR),
                                 PREFIX_QTL = c("", "", "GENOA_", "GENOA_", "met-d-", "allchr.pairseQTLTRUE.more.galasage.", "allchr.pairseQTLTRUE.more.galasage.", "allchr.pairseQTLTRUE.more.galasage.", "2019-12-11-cis-eQTLsFDR0.05-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.", "female_male_combined_pQTL_FDR5p_", "female_male_combined_eQTL_FDR5p_", "Supplementary_Sun2023_pQTL_UKB_ST9_sig", "Supplementary_Sun2023_pQTL_UKB_ST11_sig", "Supplementary_Sun2023_pQTL_UKB_ST11_sig", "tiger_eqtl_stats_allchr_withfreq_fdr5p_plus_rsid_"),
                                 SUFFIX_QTL = c(".PHENO1.glm.linear.gz", ".PHENO1.glm.linear.gz", "_summary_statistics.txt.gz", "_summary_statistics.txt.gz", ".vcf.gz", ".hg38.freqALT.txt.gz", ".hg38.freqALT.txt.gz", ".hg38.txt.gz", ".txt.gz", ".txt", ".txt", "_cisOnly.txt", "_cisOnly.txt", "_cisOnly.txt", ".tsv.gz"))


In [ ]:
#Number of cases and controls in each of the ancestries (numbers obtained from Suzuki et al. 2024)
info.T2DGGI <- data.frame(ancestry = c("AFR", "EAS", "EUR", "AMR", "SAS"),
                         n.cases = c(50251, 88109, 242283, 29375, 16832),
                         n.controls = c(103909, 339395, 1569734, 59368, 33767))

# Main function

In [ ]:
# proceed only if MR-IVW or Wald ratio's pval < 0.05 to reduce computational time
#For eQTLGen, files were already filtered to consider only genes with FDR p<5%
Run.PWCoco <- function(file_mr_res, outcome_name = "T2D", sumstatsfile, col.sumstats, pval.thr.qtl, window.cisQTL, eQTLGen = F){
    #file_mr_res: results of MR, should be in pathMR_res
    #sumstatsfile: file with summary stats of QTL
    #col.sumstats should be a vector with the old names, and the names of the vector should be:
    #TRAIT, rsid, freq, pval, beta, se , chr, pos, REF, ALT
    #pval.thr.qtl: can be single value, or a df with a threshold for each region with colnames TRAIT and pval.thr
    #window.cisQTL: whether to filter based on a window around QTL, if cisQTL provided: keep everything
    mr_res <- fread(paste0(pathMR_res, file_mr_res))

    sum2 <- subset(mr_res, method == 'Inverse variance weighted' | method == 'Wald ratio')
    sum2$shortname <- unlist(lapply(strsplit(sum2$exposure, "__"), function(x)x[5]))
    #Need to put type of QTL to lower
    exposure <- paste0(tolower(strsplit(sum2$exposure[[1]], "__")[[1]][1]), "_", paste0(strsplit(sum2$exposure[[1]], "__")[[1]][2:4], collapse = "_"))

    #Run PWCoco only for nominally significant signals to gain in computation time
    if(any(sum2$p_value < 0.05)){
        sum2.sig <- subset(sum2, p_value < 0.05)
        print(paste0('proceed to pwcoco for ', exposure, '--', outcome_name))
        # create each analyte's dir
        output_dir <- paste0(pathMR, 'pwcoco/pwcoco_res/', exposure, '/')
        system(paste0('mkdir -p ', paste0(output_dir, exposure, '--', outcome_name)))

        #Information on dataset to keep
        info.datasets.forMR.tmp <- subset(info.datasets.forMR, paste0(tolower(QTL_TYPE), "_", STUDY, "_", TISSUE, "_", ANCESTRY) == exposure)

        ########### (ii) corresponding cis_sumstats ###############
        cis_sumstats <- fread(sumstatsfile, tmpdir=paste0(pathMR, "multi_omics_github/tmpdata/"))
        cis_sumstats <- setnames(cis_sumstats, old = col.sumstats, new = names(col.sumstats))

        #Need to keep only what is in cisfile for eQTLGen as was splitted
        if(eQTLGen){
            sum2.sig <- subset(sum2.sig, shortname %in% cis_sumstats$TRAIT)
        }
        # extract the IV with the lowest pval (lead cis-pQTL) from harmonized data
        cis_sumstats_all <- list()
        cat("Getting QTL files\n")
        for(analyte in sum2.sig$shortname){
            cat(analyte, "\n")
            harmonize_df <-  fread(paste0(pathMR, 'harmonizedMR/', exposure, "/", exposure, '_T2D_harmonized_', analyte, ".txt"), fill = T)
            lead_cisqtl_rsid <- unlist(harmonize_df[order(harmonize_df$pval.exposure), "SNP"])[1] #If using rs

            # No need of modification as file read from input of MR - be careful if all and not just significant ones are in the file

            # now, extract the lead cis-pQTL position
            lead_cisqtl_chr <- unlist(cis_sumstats[TRAIT == analyte & rsid ==  lead_cisqtl_rsid,"chr"])
            lead_cisqtl_pos <- unlist(cis_sumstats[TRAIT == analyte & rsid ==  lead_cisqtl_rsid,"pos"])

            if(window.cisQTL){
                cis_sumstats_sel <- cis_sumstats[TRAIT == analyte & chr == lead_cisqtl_chr & pos >= lead_cisqtl_pos-500000 & pos <= lead_cisqtl_pos+500000,]
            }else{
                cis_sumstats_sel <- cis_sumstats[TRAIT == analyte & chr == lead_cisqtl_chr,]
            }


            # give varbeta
            cis_sumstats_sel$varbeta = cis_sumstats_sel$se^2

            # remove variants w/o rsid
            cis_sumstats_rsid <- cis_sumstats_sel[!is.na(rsid),]
            # remove duplicated snp
            cis_sumstats_nodup <- cis_sumstats_rsid[!duplicated(rsid),]

            # rename
            cis_sumstats_nodup <- setnames(cis_sumstats_nodup, old = c("chr", "TRAIT", "pos", "rsid", "ALT", "REF", "freq", "beta", "se", "pval", "varbeta"),
                                          new = c("chr", "GENE", "position", "snp", "ALT", "REF", "MAF", "beta", "SE", "pvalues", "varbeta"))

            # rename to cis_sumstats_nodup and add id column
            cis_sumstats_nodup$id <- paste('chr', cis_sumstats_nodup$chr, cis_sumstats_nodup$position, cis_sumstats_nodup$REF, cis_sumstats_nodup$ALT, sep = ":")

            # drop_na
            cis_sumstats_nodup <- cis_sumstats_nodup[complete.cases(cis_sumstats_nodup), ]

            # deal w/ extreme MAF and pval
            cis_sumstats_nodup$MAF[cis_sumstats_nodup$MAF == 0] <- 1e-300
            cis_sumstats_nodup$MAF[cis_sumstats_nodup$MAF == 1] <- 0.9999

            cis_sumstats_nodup$pvalues[cis_sumstats_nodup$pvalues == 0] <- 1e-300
            cis_sumstats_nodup$pvalues[cis_sumstats_nodup$pvalues == 1] <- 0.9999

            cis_sumstats_all <- c(cis_sumstats_all, list(cis_sumstats_nodup))
        }


        #####################
        # outcome GWAS
        #####################
        outcome <- fread(paste0(path_T2DGGI, info.datasets.forMR.tmp$ANCESTRY_T2D, "_MetalFixed_LDSC-CORR_Results1TBL_rsid.gz"), fill = T, tmpdir=paste0(pathMR, "multi_omics_github/tmpdata/"))
        #Modify the column names based on the build
        if(info.datasets.forMR.tmp$BUILD == 37){
            old.names <- c("CHR_HG19", "POS_HG19", "Effect", "StdErr", "Allele2", "Allele1", "P-value", "Freq1", "RSID")
        }else{
            old.names <- c("CHR_HG38", "POS_HG38", "Effect", "StdErr", "Allele2", "Allele1", "P-value", "Freq1", "RSID")
        }
        outcome_rename0 <- setnames(outcome, old = old.names,
                                     new = c("chr", "position", "beta", "SE", "ALT", "REF", "pvalues",  "MAF", "snp"))

        outcome_rename0$chr <- as.integer(sub("chr", "", outcome_rename0$chr))
        # keep SNPs that are present in cis_sumstats
        outcome_rename <- subset(outcome_rename0,snp %in% unlist(lapply(cis_sumstats_all, function(z) z$snp)))  # to reduce computational time, restrict SNPs here
        rm(outcome_rename0, outcome) ; gc()

        outcome_rename$id = paste0("chr", outcome_rename$chr, ':', outcome_rename$position, ':', toupper(outcome_rename$REF), ':', toupper(outcome_rename$ALT)) #ALT = effect_allele, REF = other_allele,

        # give varbeta
        outcome_rename$varbeta <- (outcome_rename$SE)^2

        # remove variants w/o rsid
        outcome_rename <- subset(outcome_rename, !is.na(snp))

        # remove duplicated id
        outcome_nodup <- subset(outcome_rename , !duplicated(snp))

        # deal w/ extreme MAF and pval to avoid errors
        outcome_nodup$MAF[outcome_nodup$MAF == 0] <- 1e-4
        outcome_nodup$MAF[outcome_nodup$MAF == 1] <- 0.9999

        outcome_nodup$pvalues[outcome_nodup$pvalues == 0] <- 1e-300
        outcome_nodup$pvalues[outcome_nodup$pvalues == 1] <- 0.9999

        ####################
        # keep only common SNPs
        # inner_join will not only keep rows with common SNPs but make their order same in both datasets
        ####################
        for(analyte in 1:length(cis_sumstats_all)){
            analyte.name = cis_sumstats_all[[analyte]]$GENE[1]
            if(!file.exists(paste0(output_dir, analyte.name, '--', outcome_name, '/', analyte.name, '--', outcome_name, ".coloc"))){
                common <- merge(cis_sumstats_all[[analyte]], outcome_nodup, by = "snp", suffix = c('.cis_sumstats', '.outcome'))
                common$ALT <- common$ALT.cis_sumstats
                common$REF <- common$REF.cis_sumstats
                common[, c("ALT.outcome","REF.outcome"):=NULL]
                if(nrow(common) == 0){
                    print('no common snp')
                }else{
                    print('common snps exist')
                    # molecularQTL
                    cis_sumstats_common <- common[,c("chr.cis_sumstats", "position.cis_sumstats", "ALT", "REF", "beta.cis_sumstats",  "pvalues.cis_sumstats", "MAF.cis_sumstats", "varbeta.cis_sumstats", "SE.cis_sumstats", "snp")]
                    colnames(cis_sumstats_common) <- c("chr", "position", "ALT", "REF", "beta", "pvalues", "MAF", "varbeta", "SE", "snp")

                    # make sure to obtain chr and pos derives from cis_sumstats (not outcome, which can be based on GRCh37/38)
                    outcome_common <- common[,c("chr.cis_sumstats", "position.cis_sumstats", "ALT", "REF", "beta.outcome", "pvalues.outcome", "MAF.outcome", "varbeta.outcome", "SE.outcome", "snp")]
                    colnames(outcome_common) <- c("chr", "position", "ALT", "REF", "beta", "pvalues", "MAF", "varbeta", "SE", "snp")

                    # run pwcoco only if there are SNPs with pval <0.05 in both traits to reduce computational time
                    if(min(cis_sumstats_common$pvalues, na.rm = T) < 0.05 & min(outcome_common$pvalues, na.rm = T) < 0.05){
                        ######
                        # run pwcoco
                        ######
                        sumstats1 <- cis_sumstats_common
                        sumstats2 <- outcome_common

                        ##################
                        # molecularQTL
                        ##################
                        sumstats1_pwcoco <- sumstats1[,c("snp", "ALT", "REF", "MAF", "beta", "SE", "pvalues")]
                        sumstats1_pwcoco$n = info.datasets.forMR.tmp$N
                        sumstats1_name <- paste0(output_dir, analyte.name, '--', outcome_name, '/', analyte.name, '--', outcome_name, '--sumstats1.tsv')
                        system(paste0('mkdir -p ', paste0(output_dir, analyte.name, '--', outcome_name)))
                        fwrite(sumstats1_pwcoco, file = sumstats1_name, sep = "\t")

                        ##################
                        # Outcome
                        ##################
                        sumstats2_pwcoco <- sumstats2[,c("snp", "ALT", "REF", "MAF", "beta", "SE", "pvalues")]
                        sumstats2_pwcoco$n = subset(info.T2DGGI, ancestry == info.datasets.forMR.tmp$ANCESTRY_T2D)$n.cases+subset(info.T2DGGI, ancestry == info.datasets.forMR.tmp$ANCESTRY_T2D)$n.controls
                        sumstats2_pwcoco$case = subset(info.T2DGGI, ancestry == info.datasets.forMR.tmp$ANCESTRY_T2D)$n.cases
                        sumstats2_name <-paste0(output_dir, analyte.name, '--', outcome_name, '/', analyte.name, '--', outcome_name, '--sumstats2.tsv')
                        fwrite(sumstats2_pwcoco, file = sumstats2_name, sep = "\t")

                        # Now both dataset 1 and 2 are ready, so let's perform pwcoco (instead of coloc)
                        # pwcoco
                        #P-value threshold
                        if(is.data.frame(pval.thr.qtl)){ qtl.thr <- subset(pval.thr.qtl, TRAIT == analyte.name)$pval.thr }
                        else{ qtl.thr <- pval.thr.qtl }
                        system(paste0(pwcoco.path, ' --bfile ', ref.bfile.path, info.datasets.forMR.tmp$ANCESTRY_T2D, ' --sum_stats1 ',  sumstats1_name,  ' --sum_stats2 ', sumstats2_name, ' --top_snp 5 --maf 0.01 --p_cutoff1 ', qtl.thr, ' --p_cutoff2 5e-8 --out ', output_dir, analyte.name, '--', outcome_name, '/', analyte.name, '--', outcome_name))
                        print('pwcoco completed')
                      }
                }
            }

        }
    }
}



## Apply on different datasets

In [ ]:
## Apply on GALAII
pval.thr.GALAII.MX <- fread(cmd = "zcat allchr.allcisQTL.more.galasage.MX.hg38.txt.gz | cut -f1,11 | sed '1d' | uniq")
colnames(pval.thr.GALAII.MX) <- c("TRAIT", "pval.thr")
pval.thr.GALAII.PR <- fread(cmd = "zcat allchr.allcisQTL.more.galasage.PR.hg38.freqALT.txt.gz | cut -f1,11 | sed '1d' | uniq")
colnames(pval.thr.GALAII.PR) <- c("TRAIT", "pval.thr")
colnames.GALAII <- c(TRAIT = "gene_id", rsid =  "variant_id", freq = "freqALT", pval = "pval_nominal", beta = "slope", se = "slope_se", chr="chr", pos = "pos", REF = "ref", ALT = "alt")


In [ ]:
res.pwcoco <- Run.PWCoco(file_mr_res = "mr_results_eqtl_GALAII_plasma_PR.txt", sumstatsfile = "allchr.allcisQTL.more.galasage.PR.hg38.freqALT.txt.gz", col.sumstats = colnames.GALAII, pval.thr.qtl = pval.thr.GALAII.PR, window.cisQTL = F)

In [ ]:
## Apply on GENOA AA
colnames.GENOA <- c(TRAIT = "GENE", rsid =  "rs", freq = "af", pval = "p_wald", beta = "beta", se = "se", chr="chr", pos = "ps", REF = "allele0", ALT = "allele1")

res.pwcoco <- Run.PWCoco(file_mr_res = "mr_results_eqtl_GENOA_plasma_AA.txt", sumstatsfile = "GENOA_AA_summary_statistics.txt.gz", col.sumstats = colnames.GENOA, pval.thr.qtl = 5e-4, window.cisQTL = F)
res.pwcoco <- Run.PWCoco(file_mr_res = "mr_results_eqtl_GENOA_plasma_EA.txt", sumstatsfile = "GENOA_EA_summary_statistics.txt.gz", col.sumstats = colnames.GENOA, pval.thr.qtl = 5e-4, window.cisQTL = F)

In [ ]:
####eQTLGen
#Only run on results FDR-significant in MR
MR.eQTLGen <- read.table(paste0(pathMR_res, "mr_results_eqtl_eQTLGen_plasma_EUR.txt"), header = T, as.is = T, sep = "\t")
#Compute the adjusted p-value with IVW and Wald Ratios
MR.eQTLGen.IVW.WR <- subset(MR.eQTLGen, method %in% c("Inverse variance weighted", "Wald ratio"))
MR.eQTLGen.IVW.WR$adj_p <- p.adjust(MR.eQTLGen.IVW.WR$p_value, method = "fdr")
head(MR.eQTLGen.IVW.WR)
#Number of FDR-significant genes
sum(MR.eQTLGen.IVW.WR$adj_p<0.05, na.rm = T)
#Only select these from the summary stats files
genes.tokeep <- gsub(unique(subset(MR.eQTLGen.IVW.WR, adj_p<0.05)$exposure), pattern = "eQTL__eQTLGen__plasma__EUR__", replacement = "")
#Split in 10 different files to parallelize
genes.tokeep.chunks <- split(genes.tokeep, ceiling(seq_along(genes.tokeep)/(length(genes.tokeep)/20)))
mclapply(1:length(genes.tokeep.chunks), function(z) system(paste0("zcat ", pathMR_eQTLGen, "2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.txt.gz | grep -E '", paste(c("Gene", paste(genes.tokeep.chunks[[z]], collapse = "|")), collapse = "|"), "' | bgzip > ", pathMR_eQTLGen, "split_by_chunks/2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.PassingMRFDR_chunk", z, ".txt.gz")), mc.cores = 20)


In [ ]:
## Apply on eQTLGen
#We will here use the adjusted p-values as there is no significance threshold given to define significance
colnames.eQTLGen <- c(TRAIT = "Gene", rsid =  "SNP", freq = "Freq", pval = "FDR", beta = "beta", se = "se", chr="SNPChr", pos = "SNPPos", REF = "OtherAllele", ALT = "AssessedAllele")
res.pwcoco <- mclapply(1:20, function(z) Run.PWCoco(file_mr_res = "mr_results_eqtl_eQTLGen_plasma_EUR.txt", sumstatsfile = paste0(pathMR_eQTLGen, "split_by_chunks/2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.PassingMRFDR_chunk", z, ".txt.gz"), col.sumstats = colnames.eQTLGen, pval.thr.qtl = 0.05, window.cisQTL = F, eQTLGen = T), mc.cores = 20)
res.pwcoco <- mclapply(c(7, 20), function(z) Run.PWCoco(file_mr_res = "mr_results_eqtl_eQTLGen_plasma_EUR.txt", sumstatsfile = paste0(pathMR_eQTLGen, "split_by_chunks/2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.PassingMRFDR_chunk", z, ".txt.gz"), col.sumstats = colnames.eQTLGen, pval.thr.qtl = 0.05, window.cisQTL = F, eQTLGen = T), mc.cores = 2)
#Rerun for missing runs
Run.PWCoco(file_mr_res = "mr_results_eqtl_eQTLGen_plasma_EUR.txt", sumstatsfile = paste0(pathMR_eQTLGen, "2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.WithBetaSe.EUR.txt.gz"), col.sumstats = colnames.eQTLGen, pval.thr.qtl = 0.05, window.cisQTL = F, eQTLGen = F)

# TIGER
We need rsIDs to use PWCoCo. We will combine the TIGER data with T2DGGI snps since only those ones can be used in coloc anyway.

In [ ]:
if(FALSE){
    TIGER_data = fread(paste0(pathMR, "tiger_eqtl_stats_allchr_withfreq.tsv.gz"), select = c("Gene", "Chrom", "Position", "Effect_allele", "Non-effect_allele", "p-value", "AF_EA", "beta", "se"), tmpdir=paste0(pathMR, "multi_omics_github/tmpdata/"))
    TIGER_data[, id:=paste(Chrom, Position, sep=":")]
    t2d = fread(paste0(pathMR, "EUR_MetalFixed_LDSC-CORR_Results1TBL_rsid.gz"), tmpdir=paste0(pathMR, "multi_omics_github/tmpdata/"))
    t2d[, id:=paste(as.integer(sub("chr", "", CHR_HG19)), POS_HG19, sep=":")]
    TIGER_data[,EA:=toupper(Effect_allele)]
    colnames(TIGER_data)[which(colnames(TIGER_data) == "Non-effect_allele")] <- "NonEffect_allele"
    TIGER_data[,NEA:=toupper(NonEffect_allele)]
    #Compute the FDR p-value
    TIGER_data[,fdr_p:=p.adjust(TIGER_data[["p-value"]], method = "fdr")]
    TIGER_t2d = merge(TIGER_data, t2d[, .(rs_id=RSID, id, effect_allele=toupper(Allele1), other_allele=toupper(Allele2))], by="id", sort = F)

    TIGER_t2d[, keep:=ifelse((EA==effect_allele & NEA==other_allele) | (EA==other_allele & NEA==effect_allele), TRUE, FALSE)]
    TIGER_t2d=TIGER_t2d[keep==TRUE]
    #Only keep TIGER information
    TIGER_t2d[, effect_allele:=NULL]
    TIGER_t2d[, other_allele:=NULL]
    TIGER_t2d[, Effect_allele:=NULL]
    TIGER_t2d[, NonEffect_allele:=NULL]
    TIGER_t2d[, keep:=NULL]
    fwrite(TIGER_t2d, paste0(path_TIGER_eQTL_EUR, "tiger_eqtl_stats_allchr_withfreq_plus_rsid_EUR_varT2DGGI.tsv.gz"),sep = "\t")
}


In [ ]:
colnames.TIGER <- c(TRAIT = "Gene", rsid =  "rs_id", freq = "AF_EA", pval = "fdr_p", beta = "beta", se = "se", chr="Chrom", pos = "Position", REF = "NEA", ALT = "EA")
res.pwcoco <- Run.PWCoco(file_mr_res = "mr_results_eqtl_TIGER_PancreaticIslets_EUR.txt", sumstatsfile = paste0(path_TIGER_eQTL_EUR, "tiger_eqtl_stats_allchr_withfreq_plus_rsid_EUR_varT2DGGI.tsv.gz"), col.sumstats = colnames.TIGER, pval.thr.qtl = 0.05, window.cisQTL = F)
